In [28]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
import time
import requests
import io
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from hmmlearn.hmm import GaussianHMM


In [29]:
df = pd.read_csv('sp500.csv')
print(df.shape)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df.head()

(6037, 7)


,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2000-01-03 00:00:00+00:00,1455.219971,1455.219971,1478.000000,1438.359985,1469.250000,931800000
2000-01-04 00:00:00+00:00,1399.420044,1399.420044,1455.219971,1397.430054,1455.219971,1009000000
2000-01-05 00:00:00+00:00,1402.109985,1402.109985,1413.270020,1377.680054,1399.420044,1085500000
2000-01-06 00:00:00+00:00,1403.449951,1403.449951,1411.900024,1392.099976,1402.109985,1092300000
2000-01-07 00:00:00+00:00,1441.469971,1441.469971,1441.469971,1400.729980,1403.449951,1225200000


In [30]:
# Trier les données par date dans l'ordre décroissant
# df = df.sort_values(by='Date', ascending=False).reset_index(drop=True)
# print(df.head())

In [34]:
# traitement des données manquantes
df = df.dropna()

# Crée et entraîne le modèle HMM
model = GaussianHMM(n_components=10, covariance_type="full", n_iter=100000)
model.fit(df['Close'].values.reshape(-1, 1))


# Crée un graphique pour chaque état caché
hidden_states = model.predict(df['Close'].values.reshape(-1, 1))

# Crée une prédiction de prix
futur = 365
pred = model.sample(n_samples=futur, currstate=hidden_states[-1])

# Tracer les données historiques
plt.figure(figsize=(14, 7))
plt.plot(df['Close'], label='Close Price')

# Plage d'indices pour les données futures
last_index = df.index[-1]
future_index = pd.date_range(start=last_index, periods=futur + 1, freq='B')[1:]

# Tracer les données futures
plt.plot(future_index, pred[0], label='Predicted Price', color='r')

# Afficher le graphique
plt.legend()
plt.show()

# Zoom sur les données futures
plt.figure(figsize=(14, 7))
plt.plot(future_index, pred[0], label='Predicted Price', color='r')
plt.legend()
plt.show()


Fitting a model with 10199 free scalar parameters with only 6037 data points will result in a degenerate solution.
c:\Users\palex\OneDrive - Marlink\Desktop\code\EPSI code\Cours\machineLearning\app-venv\Lib\site-packages\hmmlearn\hmm.py:352: RuntimeWarning: invalid value encountered in divide
  self.means_ = ((means_weight * means_prior + stats['obs'])
Some rows of transmat_ have zero sum because no transition from the state was ever observed.


ValueError: array must not contain infs or NaNs